In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import akshare as ak
import jieba
import wordcloud
from nltk.corpus import stopwords 

In [ ]:
#绘制中国疫情趋势图
chinese_total = ak.covid_19_163(indicator="中国历史累计数据")
chinese_total.eval('confirm1 = confirm - heal - dead' , inplace=True)#数据预处理
plt.figure(figsize=(10,6))
plt.rc('font',family='SimHei',size=10)#设置字体和大小，否则中文无法显示
ax1=plt.subplot(2,1,1)
confirm=chinese_total.confirm1
confirm.plot(label='现存确诊',ax=ax1)
ax1.legend()
ax1.set_title('我国现存病例趋势图')
chinese_time = ak.covid_19_163(indicator="中国历史时点数据")#每日新增
ax7=plt.subplot(2,1,2)
confirm1=chinese_time.confirm
confirm1.plot(label='每日新增确诊',ax=ax7)
ax7.legend()
ax7.set_title('我国新增病例趋势图')
plt.savefig('D:\\china_trend.png')

In [ ]:
#绘制中国现在疫情地区分布情况（高危地区）
province = ak.covid_19_dxy(indicator="中国疫情分省统计详情")
province2=province.nlargest(10,'现存确诊',keep='all')#取出现存确诊人数最多的10个省份并排名
province2=province2.sort_index(ascending=False)#将排名倒序
plt.figure(figsize=(10,6))
plt.rc('font',family='Simhei',size=10.5)
ax2=plt.subplot(1,1,1)
data=province2['现存确诊']#数据
label=province2['地区']#图例
rect=ax2.barh(range(len(label)),data,color='#03A2FF',alpha=1)
ax2.set_yticks(range(len(label)))
ax2.set_yticklabels(label)
ax2.set_title('当今中国疫情地区风险情况')
for x,y in zip(data,range(len(label))):#添加数据标签
    ax2.text(x,y,'{}'.format(x),ha='left',va='center')
plt.savefig('D:\\china_area.png')

In [ ]:
#境外输入情况分析
input_= ak.covid_19_163(indicator="境外输入确诊病例来源")
input_source=input_.groupby('source').sum()#对重复的来源地数据进行合并
input_source=input_source.nlargest(20,'value',keep='all')#取出现存确诊人数最多的10个省份并排名
input_source=input_source.sort_values('value',ascending=True)#排序
input_target=input_.groupby('target').sum()#对重复的目标地数据进行合并
input_target=input_target.nlargest(20,'value',keep='all')#取出现存确诊人数最多的10个省份并排名
input_target=input_target.sort_values('value',ascending=True)#排序
plt.figure(figsize=(10,6))
plt.rc('font',family='Simhei',size=10.5)
ax3=plt.subplot(1,2,1)
ax4=plt.subplot(1,2,2)
data=input_source.value#数据
label=input_source.index#图例
data_=input_target.value#数据
label_=input_target.index#图例
rect=ax3.barh(range(len(label)),data,color='#03A2FF',alpha=1)
rect=ax4.barh(range(len(label_)),data_,color='#03A2FF',alpha=1)
ax3.set_yticks(range(len(label)))
ax3.set_yticklabels(label)
ax3.set_title('境外输入来源国排名')
ax4.set_yticks(range(len(label_)))
ax4.set_yticklabels(label_)
ax4.set_title('境外输入目标地排名')
for x,y in zip(data,range(len(label))):#添加数据标签
    ax3.text(x,y,'{}'.format(x),ha='left',va='center')
    ax4.text(x,y,'{}'.format(x),ha='left',va='center')
plt.savefig('D:\\input.png')

In [ ]:
#分析新增境外输入疫情趋势
input_trend = ak.covid_19_163(indicator="境外输入疫情趋势")
plt.figure(figsize=(10,6))
plt.rc('font',family='SimHei',size=10)#设置字体和大小，否则中文无法显示
ax5=plt.subplot(1,2,1)
ax6=plt.subplot(1,2,2)
input_trend_add=input_trend['境外输入新增确诊']
input_trend_total=input_trend['境外输入累计确诊']
input_trend_add.plot(label='境外输入新增确诊',ax=ax5)
input_trend_total.plot(label='境外输入新增确诊',ax=ax6)
ax5.set_title('境外输入新增确诊')
ax6.set_title('境外输入累计确诊')
plt.savefig('D:\\input_trend.png')

In [ ]:
#分析疫情期间人口迁徙情况
move_in = ak.covid_19_baidu(indicator="热门迁入地")
move_in=move_in.nlargest(10,'value',keep='all')
plt.figure(figsize=(20,6))
plt.rc('font',family='Simhei',size=10.5)
ax8=plt.subplot(1,2,1)
data=move_in['value']#数据
label=move_in['city_name']#图例
rect=ax8.bar(range(len(label)),data,color='#03A2FF',alpha=1)
ax8.set_xticks(range(len(label)))
ax8.set_xticklabels(label)
ax8.set_ylabel('迁入比例')
ax8.set_title('疫情期间人口迁入地区排名')

move_out = ak.covid_19_baidu(indicator="热门迁出地")
move_out=move_out.nlargest(10,'value',keep='all')
ax9=plt.subplot(1,2,2)
data_=move_out['value']#数据
label_=move_out['city_name']#图例
rect=ax9.bar(range(len(label_)),data_,color='#03A2FF',alpha=1)
ax9.set_xticks(range(len(label_)))
ax9.set_xticklabels(label_)
ax9.set_ylabel('迁出比例')
ax9.set_title('疫情期间人口迁出地区排名')
plt.savefig('D:\\move.png')

In [ ]:
#下面进行舆情分析

In [ ]:
#专家观点关键词
quanwei = ak.covid_19_163(indicator="权威发布")
del quanwei['img']
del quanwei['time']
del quanwei['link']
del quanwei['title']
t= str(quanwei)
ls = jieba.lcut(t)
string = " ".join(ls)
w = wordcloud.WordCloud(width=1000,
                        font_path="msyh.ttc",
                        height=700,
                        background_color="white",
                        prefer_horizontal=100,
                        max_font_size=80)
w.generate(string)
w.to_file("quanwei.png")


In [ ]:
#疫情学术进展
academic = ak.covid_19_163(indicator="疫情学术进展")#疫情期间学术方面的成果
del academic['url3g']
del academic['docid']
del academic['lspri']
del academic['finearticle']
del academic['url']
del academic['quality']
del academic['imgsrc3gtype']
del academic['topicid']
del academic['stitle']
del academic['nickname']
del academic['imgsrc3g']
del academic['imgsrc']
del academic['ptime']
del academic['daynum']
del academic['extraField1']
del academic['extraField3']
t= str(academic)#把dataframe转成string格式
ls = jieba.lcut(t)#分词
string = " ".join(ls)
w = wordcloud.WordCloud(width=1000,
                        font_path="msyh.ttc",
                        height=700,
                        background_color="white",
                        prefer_horizontal=100,
                        stopwords={"zxli","通讯","作者","通讯","杂志","院士","论文"} )
w.generate(string)
w.to_file("academic.png")
